In [1]:
%pylab inline
import pandas as pd
import re
plt.rcParams['font.sans-serif']=['SimHei']

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('ftx.csv')

In [3]:
def extract_info(s, pattern):
    result = re.findall(pattern, s)
    if len(result) > 0:
        return result[0]
    else:
        return np.nan
    
def split_house_info(s):
    infos = s.split(',')
    result = '|'
    for info in infos:
        result = result + info.strip() + '|'
    return result

In [4]:
df['info'].fillna('', inplace=True)
df['info'] = df['info'].apply(split_house_info)

In [5]:
df['area'].fillna('', inplace=True)
df['unit_price'].fillna('', inplace=True)
df['area'] = df.apply(lambda row: extract_info(row['area'], r'\d+(?=㎡)'), axis=1).astype(int, errors='ignore')
df['unit_price'] = df.apply(lambda row: extract_info(row['unit_price'], r'\d+(?=元)'), axis=1).astype(int, errors='ignore')

In [6]:
df['rooms'] = df.apply(lambda row: extract_info(row['info'], r'\d(?=室)'), axis=1).astype(float)
df['halls'] = df.apply(lambda row: extract_info(row['info'], r'\d(?=厅)'), axis=1).astype(float)
df['years'] = df.apply(lambda row: extract_info(row['info'], r'建筑年代：(\d+)'), axis=1).astype(int, errors='ignore')

In [7]:
df['url'] = 'http://esf.nanjing.fang.com' + df['url']
df['loc_url'] = 'http://esf.nanjing.fang.com' + df['loc_url']

In [8]:
df2 = df.reindex(columns=['area', 'price', 'unit_price', 'rooms', 'halls', 'years',
                    'loc', 'loc_detail', 'title', 'url', 'info', 'loc_url', 'pub_time',
       'tags'])

## 把提取过的信息保存到excel

In [9]:
df2.to_excel('ftx.xlsx')

## 进行筛选

In [14]:
df2[(df2['price'] > 200) & (df2['price'] < 400) & (df2['unit_price'] < 40000)].sort_values(by='unit_price', ascending=False)

ValueError: Cannot convert non-finite values (NA or inf) to integer